## Importing required libraries

In [ ]:
import pandas as pd 
import numpy as np
import scipy
import math
from math import sqrt
import os
import tensorflow as ts
import matplotlib.pyplot as plt
import seaborn as sns
from numpy import array 
from collections import Counter
import string
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk import pos_tag
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
import math
from collections import Counter
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from nltk.probability import FreqDist
from sklearn.metrics import mean_squared_error

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


## Importing the dataset

In [ ]:
# constants for test / train / trial data
TEST = 'TEST'
TRAIN = 'TRAIN'
TRIAL = 'TRIAL'

In [ ]:
train_data = []
test_data = []
trial_data = []
data=[]

In [ ]:
df_dev=pd.read_csv('sts_dev.csv',error_bad_lines=False,header=None)
for j in range(len(df_dev)):
  arr=[]
  for i in range(len(df_dev.iloc[1,:].str.split("\t").to_list()[0])):
    if i>=4:
      arr.append(df_dev.iloc[j,:].str.split("\t").to_list()[0][i])
  trial_data.append(arr)
for i in range(5):
  print(trial_data[i])

FileNotFoundError: ignored

In [ ]:
df_test=pd.read_csv('sts_test.csv',error_bad_lines=False,header=None)
for j in range(len(df_test)):
  arr=[]
  for i in range(len(df_test.iloc[1,:].str.split("\t").to_list()[0])):
    if i>=4:
      arr.append(df_test.iloc[j,:].str.split("\t").to_list()[0][i])
  test_data.append(arr)
for i in range(5):
  print(test_data[i])

In [ ]:
df_test.head()

,0
0,main-captions\tMSRvid\t2012test\t0024\t2.500\t...
1,main-captions\tMSRvid\t2012test\t0033\t3.600\t...
2,main-captions\tMSRvid\t2012test\t0045\t5.000\t...
3,main-captions\tMSRvid\t2012test\t0063\t4.200\t...
4,main-captions\tMSRvid\t2012test\t0066\t1.500\t...


In [ ]:
df_train=pd.read_csv('sts_train.csv',error_bad_lines=False,header=None)
for j in range(len(df_train)):
  arr=[]
  for i in range(len(df_train.iloc[1,:].str.split("\t").to_list()[0])):
    if i>=4:
      arr.append(df_train.iloc[j,:].str.split("\t").to_list()[0][i])
  train_data.append(arr)
for i in range(5):
  print(train_data[i])

In [ ]:
train_data=pd.DataFrame(train_data[:]) 

In [ ]:
test_data=pd.DataFrame(test_data[:])
trial_data=pd.DataFrame(trial_data[:])
test_data.rename(columns={0:"sim",1:"sent_1",2:"sent_2"},inplace=True)
trial_data.rename(columns={0:"sim",1:"sent_1",2:"sent_2"},inplace=True)
train_data.rename(columns={0:"sim",1:"sent_1",2:"sent_2"},inplace=True)

In [ ]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Empty DataFrame

# Normalization of "sim" values 

In [ ]:
def normalize(df, feature_names):
    result = df.copy()
    for feature_name in feature_names:
        max_value = df[feature_name].max()
        min_value = df[feature_name].min()
        result[feature_name] = (df[feature_name] - min_value) / (max_value - min_value)
    return result

In [ ]:
test_data['sim'] = test_data['sim'].astype(float)
train_data['sim']=train_data['sim'].astype(float)
trial_data['sim']=trial_data['sim'].astype(float)

KeyError: ignored

In [ ]:
test_data = normalize(test_data,['sim'])
train_data = normalize(train_data,['sim'])
trial_data = normalize(trial_data,['sim'])

In [ ]:
#sick_train['sent_1'] = sick_train['sent_1'].str.lower()
#sick_train['sent_2'] = sick_train['sent_2'].str.lower()

In [ ]:
df1 = train_data['sent_1'].values
df1

TypeError: ignored

In [ ]:
df2 = train_data['sent_2'].values
len(df2)

4137

In [ ]:
import numpy as np
df = np.concatenate((df1,df2), axis=0)
len(df)

8274

In [ ]:
d_string = ''
for i in range(len(df)):
  s1 = df[i]
  d_string += ' '+(s1) 
#print(d1_string)

In [ ]:
d_string

' A plane is taking off. A man is playing a large flute. A man is spreading shreded cheese on a pizza. Three men are playing chess. A man is playing the cello. Some men are fighting. A man is smoking. The man is playing the piano. A man is playing on a guitar and singing. A person is throwing a cat on to the ceiling. The man hit the other man with a stick. A woman picks up and holds a baby kangaroo. A man is playing a flute. A person is folding a piece of paper. A man is running on the road. A dog is trying to get bacon off his back. The polar bear is sliding on the snow. A woman is writing. A cat is rubbing against baby\'s face. The man is riding a horse. A man pours oil into a pot. A man is playing a guitar. A panda is sliding down a slide. A woman is eating something. A woman peels a potato. The boy fell off his bike. The woman is playing the flute. A rabbit is running from an eagle. The woman is frying a breaded pork chop. A girl is flying a kite. A man is riding a mechanical bull.

In [ ]:
#tokens_TF_idf=nltk.wordpunct_tokenize(d_string)

In [ ]:
#len(tokens_TF_idf)

87314

# Preprocessing of Text

In [ ]:
def preprocess(text):
     
      updated = re.sub('<[^<]+?>', '', text) # html tags
      updated = re.sub(r'\W', ' ', str(text)) # Remove all the special characters
      updated = re.sub(r'\s+[a-zA-Z]\s+', ' ', text) # remove all single characters
      updated = re.sub(r'\^[a-zA-Z]\s+', ' ', text) # Remove single characters from the start
      updated = re.sub(r'\s+', ' ', text, flags=re.I) # substituting multiple spaces with single space
      updated = re.sub(r'^b\s+', '', text) # removing prefixed b 
      table = str.maketrans({key: None for key in string.punctuation})
      updated = text.translate(table)
      return updated


In [ ]:
import re

#d_string = [preprocess(sent) for sent in df]

d_string

" A group of kids is playing in a yard and an old man is standing in the background A group of children is playing in the house and there is no man standing in the background The young boys are playing outdoors and the man is smiling nearby The kids are playing outdoors near a man with a smile The young boys are playing outdoors and the man is smiling nearby Two dogs are fighting A brown dog is attacking another animal in front of the man in pants A brown dog is attacking another animal in front of the man in pants Nobody is riding the bicycle on one wheel A person is riding the bicycle on one wheel A person on a black motorbike is doing tricks with a jacket A man with a jersey is dunking the ball at a basketball game A man with a jersey is dunking the ball at a basketball game The player is dunking the basketball into the net and a crowd is in background Two people are kickboxing and spectators are not watching Two young women are sparring in a kickboxing fight Two young women are not

**Lemmatization**

In [ ]:
def lemmatize(d1_tokens):
    lz = WordNetLemmatizer()
    lemmatized = [lz.lemmatize(t) for t in d1_tokens]
    return list(set(lemmatized))

# Stopwords

In [ ]:
en_stop = set(nltk.corpus.stopwords.words('english'))
#from nltk.stem import WordNetLemmatizer
#stemmer = WordNetLemmatizer()

**Tokenization**

In [ ]:
tokens = [sent.split() for sent in df]
for i in range(len(tokens)):
  #tokens[i] = [stemmer.lemmatize(word) for word in tokens[i]]
  #tokens[i] = [word for word in tokens[i] if word not in en_stop]
  tokens[i] = [word for word in tokens[i] if len(word) > 2]

In [ ]:
tokens_with_stop = tokens
#tokens_without_stop = tokens

In [ ]:
#d1_lemmas = lemmatize(tokens_with_stop)
#len(d1_lemmas)

In [ ]:
#d2_lemmas = lemmatize(tokens_without_stop)
#len(d2_lemmas)

In [ ]:
STOP = set(nltk.corpus.stopwords.words("english"))

In [ ]:
import nltk
class SentenceModel:

    def __init__(self, sentence):
        self.raw = sentence
        normalized_sentence = sentence.replace("‘", "'").replace("’", "'")
        
        self.tokens = [t.lower() for t in nltk.word_tokenize(normalized_sentence)]
        self.tokens_without_stop = [t for t in self.tokens if t not in STOP]



# Created functions for Cosine Similarity and Word Movers Distance and Smooth Inverse Frequency

In [ ]:
!pip install flair

     |████████████████████████████████| 450kB 7.0MB/s 
     |████████████████████████████████| 798kB 11.0MB/s 
     |████████████████████████████████| 1.1MB 20.0MB/s 
     |████████████████████████████████| 983kB 33.8MB/s 
     |████████████████████████████████| 71kB 12.2MB/s 
     |████████████████████████████████| 19.7MB 1.3MB/s 
     |████████████████████████████████| 1.3MB 60.0MB/s 
     |████████████████████████████████| 2.9MB 50.9MB/s 
     |████████████████████████████████| 890kB 55.4MB/s 
  Created wheel for segtok: filename=segtok-1.5.10-cp36-none-any.whl size=25019 sha256=cf052b483da7da63170a267a3254ecbd527d2f1f0012d2274699072a4060c38c
  Stored in directory: /root/.cache/pip/wheels/b4/39/f6/9ca1c5cabde964d728023b5751c3a206a5c8cc40252321fb6b
  Created wheel for mpld3: filename=mpld3-0.3-cp36-none-any.whl size=116678 sha256=17a66efa2bfffff198dbc210a5aeb56491d9a26cb27aafcf894886f0b10ecb94
  Stored in directory: /root/.cache/pip/wheels/c0/47/fb/8a64f89aecfe0059830479308ad42d62e89

# Cosine Similarity

In [ ]:
from tqdm import tqdm
from flair.data import Sentence
def run_context_avg_benchmark(sentences1, sentences2, model=None, use_stoplist=False, doc_freqs=None):
    if doc_freqs is not None:
        N = doc_freqs["NUM_DOCS"]

    sims = []
    print(sims)
    for (sent1, sent2) in tqdm(zip(sentences1, sentences2), total=len(sentences1)):
      
      tokens1 = sent1.tokens_without_stop if use_stoplist else sent1.tokens
      tokens2 = sent2.tokens_without_stop if use_stoplist else sent2.tokens

      flair_tokens1 = sent1.tokens
      flair_tokens2 = sent2.tokens

      flair_sent1 = Sentence(" ".join(flair_tokens1))
      flair_sent2 = Sentence(" ".join(flair_tokens2))
      
      model.embed(flair_sent1)
      model.embed(flair_sent2)

      embeddings_map1 = {}
      embeddings_map2 = {}

      for token in flair_sent1:
        embeddings_map1[token.text] = np.array(token.embedding.data.tolist())

      for token in flair_sent2:
        embeddings_map2[token.text] = np.array(token.embedding.data.tolist())

      if len(tokens1) == 0 or len(tokens2) == 0:
        sims.append(0)
        continue

      tokfreqs1 = Counter(tokens1)
      tokfreqs2 = Counter(tokens2)

      weights1 = [tokfreqs1[token] * math.log(N / (doc_freqs.get(token, 0) + 1)) 
                  for token in tokfreqs1 if token in embeddings_map1] if doc_freqs else None
      weights2 = [tokfreqs2[token] * math.log(N / (doc_freqs.get(token, 0) + 1))
                  for token in tokfreqs2 if token in embeddings_map2] if doc_freqs else None

      embedding1 = np.average([embeddings_map1[token] for token in tokfreqs1 if token in embeddings_map1], axis=0, weights=weights1).reshape(1, -1)
      embedding2 = np.average([embeddings_map2[token] for token in tokfreqs2 if token in embeddings_map2], axis=0, weights=weights2).reshape(1, -1)

      sim = cosine_similarity(embedding1, embedding2)[0][0]
      
      sims.append(sim)
    print(sims)
    return sims

# Word Mover Distance   



In [ ]:
from sklearn import preprocessing
def run_wmd_similarity(sentences1, sentences2, model, use_stoplist=False):
    sims = []
    for (sent1, sent2) in zip(sentences1, sentences2):

        tokens1 = sent1.tokens_without_stop if use_stoplist else sent1.tokens
        tokens2 = sent2.tokens_without_stop if use_stoplist else sent2.tokens

        tokens1 = [token for token in tokens1 if token in model]
        tokens2 = [token for token in tokens2 if token in model]

        if len(tokens1) == 0 or len(tokens2) == 0:
            tokens1 = [token for token in sent1.tokens if token in model]
            tokens2 = [token for token in sent2.tokens if token in model]

        sims.append(model.wmdistance(tokens1, tokens2))
    
    
    x_array = np.array(sims)
    print(x_array)
    normalized_arr = preprocessing.normalize([x_array])
    sims=normalized_arr[0].tolist()
    return sims

    

# Smooth Inverse Frequency

In [ ]:
list1=[]
[list1.append(i) for sent_1 in tokens_with_stop for i in sent_1]
len(list1)
frequencies = FreqDist(list1)
print(frequencies)

<FreqDist with 3585 samples and 12057 outcomes>


In [ ]:
import numpy as np
from flair.data import Sentence
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity


def remove_first_principal_component(X):
    svd = TruncatedSVD(n_components=1, n_iter=7, random_state=0)
    svd.fit(X)
    pc = svd.components_
    XX = X - X.dot(pc.transpose()) * pc
    return XX

In [ ]:
def run_context_sif_benchmark(sentences1, sentences2, model, freqs={}, use_stoplist=False, a=0.001):
    total_freq = sum(freqs.values())

    embeddings = []
    count=0

    # SIF requires us to first collect all sentence embeddings and then perform
    # common component analysis.
    for (sent1, sent2) in zip(sentences1, sentences2):
        tokens1 = sent1.tokens_without_stop if use_stoplist else sent1.tokens
        tokens2 = sent2.tokens_without_stop if use_stoplist else sent2.tokens

        flair_tokens1 = sent1.tokens
        flair_tokens2 = sent2.tokens

        flair_sent1 = Sentence(" ".join(flair_tokens1))
        flair_sent2 = Sentence(" ".join(flair_tokens2))

        model.embed(flair_sent1)
        model.embed(flair_sent2)

        embeddings_map1 = {}
        embeddings_map2 = {}

        for token in flair_sent1:
            embeddings_map1[token.text] = np.array(token.embedding.data.tolist())

        for token in flair_sent2:
            embeddings_map2[token.text] = np.array(token.embedding.data.tolist())

        tokens1 = [token for token in tokens1]
        tokens2=[token for token in tokens2]
        if len(tokens1) == 0 or len(tokens2) == 0:
            embedding1 = 0
            embedding2 = 0

            embeddings.append(embedding1)
            embeddings.append(embedding2)

            #embeddings = remove_first_principal_component(np.array(embeddings))
            sims = [0]
            continue
        else:
          weights1 = [a / (a + freqs.get(token, 0) / total_freq) for token in tokens1]
          print(weights1)
          weights2 = [a / (a + freqs.get(token, 0) / total_freq) for token in tokens2]

          embedding1 = np.average([embeddings_map1[token] for token in tokens1], axis=0, weights=weights1)
          embedding2 = np.average([embeddings_map2[token] for token in tokens2], axis=0, weights=weights2)

          embeddings.append(embedding1)
          embeddings.append(embedding2)

          embeddings = remove_first_principal_component(np.array(embeddings))
          sims = [cosine_similarity(embeddings[idx * 2].reshape(1, -1),
                              embeddings[idx * 2 + 1].reshape(1, -1))[0][0]
          for idx in range(int(len(embeddings) / 2))]

    return sims

# ELMo Implementation

In [ ]:
from flair.embeddings import ELMoEmbeddings 


In [ ]:
!pip install allennlp==0.9.0

In [ ]:
elmo = ELMoEmbeddings('original')

In [ ]:
import pickle
with open('./modelElmo.pkl', 'wb') as output:
  pickle.dump(elmo, output)

In [ ]:
df_elmo=train_data

In [ ]:
df_elmo["sent_1"] = df_elmo['sent_1'].str.replace('[^\w\s]','')
df_elmo["sent_2"] = df_elmo['sent_2'].str.replace('[^\w\s]','')

In [ ]:
sentences1 = [SentenceModel(s) for s in df_elmo['sent_1']] 
sentences2 = [SentenceModel(s) for s in df_elmo['sent_2']]

#sims = run_context_avg_benchmark(sentences1, sentences2, model=elmo, use_stoplist=False, doc_freqs=None)
sims = run_context_sif_benchmark(sentences1, sentences2, model=elmo, freqs=frequencies, use_stoplist=False, a=0.001)
df_elmo["predicted_sim"] = pd.Series(sims).values

df_elmo["diff"] = np.sqrt((df_elmo["predicted_sim"] - df_elmo["sim"])**2)

#df_elmo = df_elmo.sort_values("sim", ascending=False)


In [ ]:
import scipy
from math import sqrt 
from sklearn.metrics import mean_squared_error
pearson_correlation = scipy.stats.pearsonr(sims, df_elmo['sim'])[0]
spearman_correlation = scipy.stats.spearmanr(sims, df_elmo['sim'])[0]
rmse = (mean_squared_error(sims, df_elmo['sim']))
textstr = 'RMSE=%.3f\nPearson Correlation=%.3f\nSpearman Correlation=%.3f'%(rmse, pearson_correlation, spearman_correlation)


In [ ]:
print(textstr) #Cosine

RMSE=0.137
Pearson Correlation=0.637
Spearman Correlation=0.619


In [ ]:
print(textstr) #SIF

RMSE=0.062
Pearson Correlation=0.653
Spearman Correlation=0.624


In [ ]:
#df_elmo = df_elmo.sort_values("diff", ascending=False)

In [ ]:
df_elmo[:100]

,idx,sent_1,sent_2,sim,label,predicted_sim,diff
0,1,A group of kids is playing in a yard and an ol...,A group of boys in a yard is playing and a man...,0.875,NEUTRAL,0.742519,0.132481
1,2,A group of children is playing in the house an...,A group of kids is playing in a yard and an ol...,0.550,NEUTRAL,0.579370,0.029370
2,3,The young boys are playing outdoors and the ma...,The kids are playing outdoors near a man with ...,0.925,ENTAILMENT,0.305195,0.619805
3,5,The kids are playing outdoors near a man with ...,A group of kids is playing in a yard and an ol...,0.600,NEUTRAL,0.169292,0.430708
4,9,The young boys are playing outdoors and the ma...,A group of kids is playing in a yard and an ol...,0.675,NEUTRAL,0.230414,0.444586
...,...,...,...,...,...,...,...
95,241,A woman is wearing an Egyptian headdress,A woman is wearing an Indian headdress,0.750,NEUTRAL,0.945695,0.195695
96,248,The black woman is wearing glasses over the he...,A woman is wearing an Egyptian headdress,0.650,NEUTRAL,0.475605,0.174395
97,249,A woman is wearing an Egyptian hat on her head,The woman is wearing glasses and a black headd...,0.375,NEUTRAL,0.418359,0.043359
98,254,A hiker is on top of the mountain and is dancing,There is no hiker dancing on top of the mountain,0.550,CONTRADICTION,0.633669,0.083669


##STS_TEST Results

In [ ]:
df1 = test_data['sent_1'].values
df1

array(['A girl is styling her hair.',
       'A group of men play soccer on the beach.',
       "One woman is measuring another woman's ankle.", ...,
       'How much do you know about Secret Service?',
       'Obama Struggles to Soothe Saudi Fears As Iran Talks Resume',
       'South Korea declares end to MERS outbreak'], dtype=object)

In [ ]:
df2 = test_data['sent_2'].values
df2

array(['A girl is brushing her hair.',
       'A group of boys are playing soccer on the beach.',
       "A woman measures another woman's ankle.", ...,
       'Lawmakers from both sides express outrage at Secret Service',
       'Myanmar Struggles to Finalize Voter Lists for Sunday Polls',
       'North Korea Delegation Meets With South Korean Officials'],
      dtype=object)

In [ ]:
import numpy as np
df = np.concatenate((df1,df2), axis=0)
len(df)

2046

In [ ]:
d_string = ''
for i in range(len(df)):
  s1 = df[i]
  d_string += ' '+(s1) 
#print(d1_string)

In [ ]:
d_string

' A girl is styling her hair. A group of men play soccer on the beach. One woman is measuring another woman\'s ankle. A man is cutting up a cucumber. A man is playing a harp. A woman is cutting onions. A man is riding an electric bicycle. A man is playing the drums. A man is playing guitar. A man is playing a guitar. A man is playing a guitar. A man is cutting an onion. A man is cycling. A man is slicing open a fish. A man is slicing a tomato. A man is playing a guitar. A baby panda goes down a slide. A man is singing and playing a guitar. A man attacks a woman. A man is driving a car. A woman is cutting tofu. The woman is styling her hair. Two zebras play in an open field. A man is cutting a potato. A man is slicing an onion. A  man is dancing. A man is riding a motorcycle. A woman is slicing garlics. A man is speaking. A little boy is singing and playing a guitar. A turtle is swimming in water. A young woman is putting stickers all over her face. A woman is wrapping tofu. A cat is ea

In [ ]:
en_stop = set(nltk.corpus.stopwords.words('english'))
#from nltk.stem import WordNetLemmatizer
#stemmer = WordNetLemmatizer()

In [ ]:
tokens = [sent.split() for sent in df]
for i in range(len(tokens)):
  #tokens[i] = [stemmer.lemmatize(word) for word in tokens[i]]
  #tokens[i] = [word for word in tokens[i] if word not in en_stop]
  tokens[i] = [word for word in tokens[i] if len(word) > 2]

In [ ]:
tokens_with_stop = tokens
#tokens_without_stop = tokens

In [ ]:
STOP = set(nltk.corpus.stopwords.words("english"))

In [ ]:
import nltk
class SentenceModel:

    def __init__(self, sentence):
        self.raw = sentence
        normalized_sentence = sentence.replace("‘", "'").replace("’", "'")
        
        self.tokens = [t.lower() for t in nltk.word_tokenize(normalized_sentence)]
        self.tokens_without_stop = [t for t in self.tokens if t not in STOP]

In [ ]:
# run cosine,wmd,sif and some extra functions

In [ ]:
pip install flair

     |████████████████████████████████| 450kB 4.2MB/s 
     |████████████████████████████████| 798kB 7.6MB/s 
     |████████████████████████████████| 19.7MB 1.2MB/s 
     |████████████████████████████████| 1.3MB 45.1MB/s 
     |████████████████████████████████| 983kB 41.8MB/s 
     |████████████████████████████████| 71kB 8.3MB/s 
     |████████████████████████████████| 1.1MB 41.3MB/s 
     |████████████████████████████████| 890kB 42.3MB/s 
     |████████████████████████████████| 2.9MB 41.2MB/s 
  Created wheel for sqlitedict: filename=sqlitedict-1.7.0-cp36-none-any.whl size=14376 sha256=39726b480670449af7cb91b77c80739d346b302c53c7057c2be56d2c2b30b13c
  Stored in directory: /root/.cache/pip/wheels/cf/c6/4f/2c64a43f041415eb8b8740bd80e15e92f0d46c5e464d8e4b9b
  Created wheel for mpld3: filename=mpld3-0.3-cp36-none-any.whl size=116678 sha256=f4a62f7d990153a6cd36818b5b116153f22b7be5fe68e52973e55285633cf5e8
  Stored in directory: /root/.cache/pip/wheels/c0/47/fb/8a64f89aecfe0059830479308ad42d

In [ ]:
from flair.embeddings import ELMoEmbeddings 

In [ ]:
!pip install allennlp==0.9.0

     |████████████████████████████████| 7.6MB 7.9MB/s 
     |████████████████████████████████| 317kB 46.9MB/s 
     |████████████████████████████████| 102kB 14.7MB/s 
     |████████████████████████████████| 51kB 8.2MB/s 
     |████████████████████████████████| 163kB 53.0MB/s 
     |████████████████████████████████| 133kB 58.2MB/s 
     |████████████████████████████████| 30.9MB 93kB/s 
     |████████████████████████████████| 266kB 55.7MB/s 
     |████████████████████████████████| 5.5MB 55.3MB/s 
     |████████████████████████████████| 51kB 9.4MB/s 
     |████████████████████████████████| 245kB 54.4MB/s 
     |████████████████████████████████| 7.2MB 47.7MB/s 
     |████████████████████████████████| 71kB 11.8MB/s 
     |████████████████████████████████| 3.2MB 47.9MB/s 
     |████████████████████████████████| 2.1MB 21.4MB/s 
     |████████████████████████████████| 92kB 13.2MB/s 
     |████████████████████████████████| 163kB 52.7MB/s 
     |████████████████████████████████| 245kB 53.8MB/s 


In [ ]:
elmo = ELMoEmbeddings('original')

100%|██████████| 374434792/374434792 [00:04<00:00, 82250387.78B/s]


In [ ]:
import pickle
with open('./modelElmo.pkl', 'wb') as output:
  pickle.dump(elmo, output)

In [ ]:
df_elmo=test_data

In [ ]:
df_elmo["sent_1"] = df_elmo['sent_1'].str.replace('[^\w\s]','')
df_elmo["sent_2"] = df_elmo['sent_2'].str.replace('[^\w\s]','')

In [ ]:
sentences1 = [SentenceModel(s) for s in df_elmo['sent_1']] 
sentences2 = [SentenceModel(s) for s in df_elmo['sent_2']]

#sims = run_context_avg_benchmark(sentences1, sentences2, model=elmo, use_stoplist=True, doc_freqs=None)
sims = run_context_sif_benchmark(sentences1, sentences2, model=elmo, freqs=frequencies, use_stoplist=True, a=0.001)
df_elmo["predicted_sim"] = pd.Series(sims).values

df_elmo["diff"] = np.sqrt((df_elmo["predicted_sim"] - df_elmo["sim"])**2)

#df_elmo = df_elmo.sort_values("sim", ascending=False)

[0.17980225778069403, 0.8577221313224729, 0.9234127288044728]
[0.23614783477290088, 0.280024154028381, 0.5011846863698715, 0.5725886878472717, 0.7508874634115962]


AttributeError: ignored

In [ ]:
import scipy
from math import sqrt 
from sklearn.metrics import mean_squared_error
pearson_correlation = scipy.stats.pearsonr(sims, df_elmo['sim'])[0]
spearman_correlation = scipy.stats.spearmanr(sims, df_elmo['sim'])[0]
rmse = (mean_squared_error(sims, df_elmo['sim']))
textstr = 'RMSE=%.3f\nPearson Correlation=%.3f\nSpearman Correlation=%.3f'%(rmse, pearson_correlation, spearman_correlation)

In [ ]:
print(textstr)

RMSE=0.068
Pearson Correlation=0.628
Spearman Correlation=0.612
